In [148]:
import clickhouse_connect
import pandas as pd
import numpy as np
import calendar
import datetime
from datetime import datetime, timedelta

client = clickhouse_connect.get_client(host='h4ht169yky.us-central1.gcp.clickhouse.cloud', port=8443, username='default', password='_J8EZmEnKUcq0')

result = client.query('SELECT * FROM dolar_act')
result.result_rows
Out[13]: [(2000, -50.9035)]

# Realizamos la importacion de la base de datos actualizada desde ClckHouse

In [149]:
client = clickhouse_connect.get_client(host='h4ht169yky.us-central1.gcp.clickhouse.cloud', port=8443, username='default', password='_J8EZmEnKUcq0')

result = client.query('SELECT * FROM dolar_act')
result.result_rows
Out[13]: [(2000, -50.9035)]

# ETL para normalizar el nombre de las columnas de la base de origen, tipos de datos y caracteres especiales.

In [150]:
Dolar = pd.DataFrame(result.result_rows)

nuevos_nombres = {
    0: 'fecha',
    1: 'hora',
    2: 'am_pm',
    3: 'blue_compra',
    4: 'oficial_compra',
    5: 'bolsa_compra',
    6: 'contado_liqui_compra',
    7: 'crypto_compra',
    8: 'blue_venta',
    9: 'oficial_venta',
    10: 'bolsa_venta',
    11: 'contado_liqui_venta',
    12: 'crypto_venta',
    13: 'solidario_venta'
}

Dolar = Dolar.rename(columns=nuevos_nombres)

Dolar['crypto_venta'] = Dolar['crypto_venta'].astype(int)

Dolar['hora_2'] = Dolar['hora'].apply(lambda x: datetime.strptime(x, '%H:%M'))

Dolar['hora_2'] = Dolar.apply(lambda row: row['hora_2'] + timedelta(hours=12) if row['am_pm'] == 'P' else row['hora_2'], axis=1)

Dolar['hora_2'] = Dolar['hora_2'].astype(str)


Dolar['dia'] = Dolar['fecha'].str[:2] #Va

Dolar['ano'] = Dolar['fecha'].str[6:] #Va
Dolar['ano'] = '20' + Dolar['ano'] #Va

Dolar['mes'] = Dolar['fecha'].str[3:5]

Dolar['fecha_ano_mes_dia'] = Dolar['ano'] + Dolar['mes'] + Dolar['dia']
Dolar['fecha_ano_mes_dia_hora'] = Dolar['fecha_ano_mes_dia'] + Dolar['hora_2'] 


Dolar = Dolar.sort_values(by='fecha_ano_mes_dia_hora', ascending=True)
Dolar.tail(1)


,fecha,hora,am_pm,blue_compra,oficial_compra,bolsa_compra,contado_liqui_compra,crypto_compra,blue_venta,oficial_venta,bolsa_venta,contado_liqui_venta,crypto_venta,solidario_venta,hora_2,dia,ano,mes,fecha_ano_mes_dia,fecha_ano_mes_dia_hora
305,05/11/23,12:51,P,905.0,348.96,839.7,856.7,864.0,925.0,368.6,844.0,840.95,875,639.63,1900-01-02 00:51:00,05,2023,11,20231105,202311051900-01-02 00:51:00


In [151]:
print(Dolar.dtypes)

fecha                      object
hora                       object
am_pm                      object
blue_compra               float64
oficial_compra            float64
bolsa_compra              float64
contado_liqui_compra      float64
crypto_compra             float64
blue_venta                float64
oficial_venta             float64
bolsa_venta               float64
contado_liqui_venta       float64
crypto_venta                int32
solidario_venta           float64
hora_2                     object
dia                        object
ano                        object
mes                        object
fecha_ano_mes_dia          object
fecha_ano_mes_dia_hora     object
dtype: object


# Proceso para forecasting:

In [152]:
x = Dolar['fecha_ano_mes_dia_hora'].values
y = Dolar['crypto_venta'].values

In [153]:
prediccion =202310311147
for i in range(0, 2):
   coef = np.polyfit(x, y, i)
   p = np.polyval(coef, prediccion )
   print(p)

TypeError: can only concatenate str (not "float") to str

In [ ]:
import calendar
import datetime

# Obtén la fecha actual
today = datetime.date.today()

# Suma 30 días a la fecha actual
date_in_30_days = today + datetime.timedelta(days=30)

print("Fecha de dentro de 30 días:", date_in_30_days)

Fecha de dentro de 30 días: 2023-12-05
